In [1]:
# mnist_cnn_keras

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)

print(tf.__version__)
print(tf.keras.__version__)

2.3.0
2.4.0


In [2]:
# MNIST Data Set
mnist = tf.keras.datasets.mnist
(x_train,y_train), (x_test,y_test) = mnist.load_data()

# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)

# X값의 shape을 4차원으로 변환
x_train = tf.reshape(x_train,[-1,28,28,1])
x_test = tf.reshape(x_test,[-1,28,28,1])

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1)
(60000,) (10000,)


In [3]:
# MNIST 분류 CNN keras 모델 구현
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28, 28, 1),kernel_size=(3,3),filters=32,padding='same'),
    tf.keras.layers.Dense(units=10,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    
    tf.keras.layers.Conv2D(input_shape=(14,14,1), kernel_size=(3,3), padding='same',filters=64),
    tf.keras.layers.Dense(units=10, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same'),

    tf.keras.layers.Conv2D(input_shape=(7,7,1), kernel_size=(3,3), padding='same',filters=1284),
    tf.keras.layers.Dense(units=10, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss= 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
# tf.keras.layers.Conv2D(
#     filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
#     dilation_rate=(1, 1), groups=1, activation=None, use_bias=True,
#     kernel_initializer='glorot_uniform', bias_initializer='zeros',
#     kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
#     kernel_constraint=None, bias_constraint=None, **kwargs
# )


# https://yeomko.tistory.com/40
# Xavier Glorot Initialization

# loss 종류
# mean_squared_error : 평균제곱 오차
# binary_crossentropy : 이진분류 오차
# categorical_crossentropy : 다중 분류 오차. one-hot encoding 클래스, [0.2, 0.3, 0.5] 와 같은 출력값과 실측값의 오차값을 계산한다.
# sparse_categorical_crossentropy: 다중 분류 오차. 위와 동일하지만 , integer type 클래스라는 것이 다르다.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
dense (Dense)                (None, 28, 28, 10)        330       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        5824      
_________________________________________________________________
dense_1 (Dense)              (None, 14, 14, 10)        650       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 10)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 1284)        1

In [4]:
model.fit(x_train,y_train,epochs=25,validation_split=0.25)

Epoch 1/25
1407/1407 [==============================] - 9s 6ms/step - loss: 0.3495 - accuracy: 0.8891 - val_loss: 0.1389 - val_accuracy: 0.9556
Epoch 2/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1446 - accuracy: 0.9555 - val_loss: 0.1199 - val_accuracy: 0.9635
Epoch 3/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1335 - accuracy: 0.9587 - val_loss: 0.1217 - val_accuracy: 0.9626
Epoch 4/25
1407/1407 [==============================] - 9s 6ms/step - loss: 0.1229 - accuracy: 0.9622 - val_loss: 0.1457 - val_accuracy: 0.9617
Epoch 5/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1091 - accuracy: 0.9666 - val_loss: 0.0936 - val_accuracy: 0.9711
Epoch 6/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1016 - accuracy: 0.9687 - val_loss: 0.1129 - val_accuracy: 0.9666
Epoch 7/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.0946 - accuracy: 0.9706 - val_loss: 0.0809 - val_accuracy:

In [5]:
model.evaluate(x_test,y_test,verbose=0)
# [0.05747390165925026, 0.9819999933242798]

[0.05747390165925026, 0.9819999933242798]

In [6]:
# VGGNet (VGG-19) 스타일의 MNIST 분류 CNN 모델 
#--------------------------------------------
# ( Conv2D * 2개  --> MaxPool2D ) * 2회 : 4층
# ( Conv2D * 4개  --> MaxPool2D ) * 3회 : 12층
# Dense * 3개                           : 3층
#--------------------------------------------
#                                     총 19층
#--------------------------------------------
# 각 네트워크마다 필터의 수를 2배로 증가 시킨다 : 32-->64-->128-->256-->512

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 256)        

In [7]:
model.fit(x_train, y_train , epochs=25, validation_split=0.25)
model.evaluate(x_test, y_test, verbose=0)
# [0.034512970596551895, 0.9904999732971191]
# [0.02834535576403141, 0.9921000003814697]  : 정확도 향상,코랩에서 실행

Epoch 1/25
1407/1407 [==============================] - 11s 8ms/step - loss: 0.6628 - accuracy: 0.8380 - val_loss: 0.0833 - val_accuracy: 0.9765
Epoch 2/25
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1739 - accuracy: 0.9508 - val_loss: 0.0664 - val_accuracy: 0.9792
Epoch 3/25
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1418 - accuracy: 0.9601 - val_loss: 0.0623 - val_accuracy: 0.9835
Epoch 4/25
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1253 - accuracy: 0.9654 - val_loss: 0.0499 - val_accuracy: 0.9863
Epoch 5/25
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1163 - accuracy: 0.9700 - val_loss: 0.0488 - val_accuracy: 0.9871
Epoch 6/25
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1058 - accuracy: 0.9724 - val_loss: 0.0572 - val_accuracy: 0.9835
Epoch 7/25
1407/1407 [==============================] - 11s 8ms/step - loss: 0.1013 - accuracy: 0.9739 - val_loss: 0.0421 - val_ac

[0.034512970596551895, 0.9904999732971191]